# GRAS Stemmer

In [ ]:
# Import required library
import os
from tqdm.notebook import tqdm
import pickle
from collections import defaultdict
from itertools import combinations

SAVE_PATH = "/content/drive/Shareddrives/IR project _ GRAS/dataset"

In [ ]:
class GRAS:
    def __init__(self,unique_tokens,l,alpha,delta):
        self.unique_tokens = unique_tokens  # list of unique tokens
        self.l = l
        self.alpha = alpha     # edge frequency must be >= alpha
        self.delta = delta     # cohesion must be >= delta
        self.sets = defaultdict(list)             # {key = l  length suffix  :=>  value = set of words contains l  length suffix }
        self.suffix_pair_freq = defaultdict(int)  # {key = tuple of prefix pair  :=>   value = frequency}
        self.root_word = dict()                   # {key = word => value = root_word}
    
    # helping functions
    def lcs(self, x, y):
        i = 0
        while i < len(x) and i < len(y) and x[i] == y[i]: i += 1
        if i == len(x):
            return ('', y[i:], i)
        if i == len(y):
            return (x[i:], '', i)
        return (x[i:], y[i:], i)

    def cohesion(self,adjp,adjv):
        deno = 1 + len(adjp.intersection(adjv))
        numo = len(adjv)
        return deno / numo

    def highest_freq_vertex(self,graph):
        max = -1
        for vertex in graph:
            if len(graph[vertex]) > max:
                vertex_name = vertex
                max = len(graph[vertex])
            elif len(graph[vertex]) == max and len(vertex)<len(vertex_name):
                vertex_name = vertex
        return vertex_name

    def deletion_vertex(self,graph, graph2, vertex_name):
        adjs =  graph2[vertex_name]
        for adj in adjs:
            for element in graph[adj]:
                if element[-1] == vertex_name:
                    temp = element
                    break
            graph[adj].remove(temp)
            graph2[adj].remove(vertex_name)
        graph.pop(vertex_name)
        graph2.pop(vertex_name)
        return graph,graph2


    # generate sets
    def get_sets(self):
        for token in tqdm(self.unique_tokens):
            if(len(token)<self.l):
                continue
            self.sets[token[:self.l]].append(token)
        
        key = list(self.sets.keys())
        for k in key:
            if len(self.sets[k]) <= 1:
                self.sets.pop(k)

    # generates suffix_pair_freq 
    def identify_suffix(self):
        for key in tqdm(self.sets):
            for i, j in combinations(self.sets[key], 2):
                (sf1, sf2, k) = self.lcs(i, j)
                edge = tuple(sorted([sf1,sf2]))
                if (edge not in self.suffix_pair_freq):
                    self.suffix_pair_freq[edge] = 1
                else:
                    self.suffix_pair_freq[edge] += 1

        key = list(self.suffix_pair_freq.keys())
        for k in tqdm(key):
            if self.suffix_pair_freq[k] < self.alpha:
                self.suffix_pair_freq.pop(k)
        
    # generates classes and root word directory    
    def class_generation(self):
        for s in tqdm(self.sets):
            graph = defaultdict(set)
            graph2 = defaultdict(set)
            for i, j in combinations(self.sets[s], 2):
                (sf1, sf2, k) = self.lcs(i, j)
                edge = tuple(sorted([sf1,sf2]))
                if (edge not in self.suffix_pair_freq):
                    continue
                else:
                    graph[i].add(tuple([self.suffix_pair_freq[edge], j]))
                    graph[j].add(tuple([self.suffix_pair_freq[edge], i]))
                    graph2[i].add(j)
                    graph2[j].add(i)
            
            while len(graph) > 0:
                high_vertex = self.highest_freq_vertex(graph)
                n = len(graph[high_vertex])
                if n > 0:
                    adjs = sorted(list(graph[high_vertex]),reverse=True)
                    for adj in adjs:
                        if  self.cohesion(graph2[high_vertex], graph2[adj[-1]]) >= self.delta:
                            self.root_word[adj[-1]] = high_vertex
                            graph,graph2 = self.deletion_vertex(graph,graph2,adj[-1])
                        else:
                            graph[high_vertex].remove(adj)
                            graph2[high_vertex].remove(adj[-1])

                            graph[adj[-1]].remove((adj[0],high_vertex))
                            graph2[adj[-1]].remove(high_vertex)
                graph.pop(high_vertex)
                graph2.pop(high_vertex)
        return self.root_word

    def get_root_dir(self):
        print("generating sets...")
        self.get_sets()
        print("generating suffix_pair_freq...." )
        self.identify_suffix()
        print("generating classes....")
        self.class_generation()
        print("root dir ready...")
        return self.root_word

In [ ]:
l = 6
alpha = 4
delta = 0.5
unique_tokens = pickle.load(open(os.path.join(SAVE_PATH,"unique_token"),'rb'))

In [ ]:
st = GRAS(unique_tokens,l,alpha,delta)
root_word = st.get_root_dir()

generating sets...



generating suffix_pair_freq....



generating classes....



root dir ready...


In [ ]:
len(root_word)

968381

In [ ]:
pickle.dump( root_word, open(os.path.join(SAVE_PATH,"root_word_dir"), "wb" ) )

In [ ]:
for k in root_word:
    print(k , ':',root_word[k])

Streaming output truncated to the last 5000 lines.
શિક્શણવર્તુળોમાં : શિક્શણ
શિક્શણફીમાં : શિક્શણ
શિક્શણધિકારીઓ : શિક્શણ
શિક્શણદાતાઓ : શિક્શણ
શિક્શણખાતાનો : શિક્શણ
શિક્શણકાર્યો : શિક્શણ
શિક્શણકલાને : શિક્શણ
શિક્શણસત્રમાં : શિક્શણ
શિક્શણશાસ્ત્રીને : શિક્શણ
શિક્શણવેરાના : શિક્શણ
શિક્શણવિભાગે : શિક્શણ
શિક્શણવિદોનું : શિક્શણ
શિક્શણવર્ગનું : શિક્શણ
શિક્શણપ્રથાના : શિક્શણ
શિક્શણપાઠો : શિક્શણ
શિક્શણધિકારીની : શિક્શણ
શિક્શણધિકારીઓને : શિક્શણ
શિક્શણક્શેત્રથી : શિક્શણ
શિક્શણસ્વાસ્થ્ય : શિક્શણ
શિક્શણસેવાના : શિક્શણ
શિક્શણસાધન : શિક્શણ
શિક્શણશાસ્ત્રીઓનો : શિક્શણ
શિક્શણવીદો : શિક્શણ
શિક્શણવિભાગમાં : શિક્શણ
શિક્શણભૂખને : શિક્શણ
શિક્શણબોર્ડે : શિક્શણ
શિક્શણપ્રેમીઓનું : શિક્શણ
શિક્શણપ્રત્યે : શિક્શણ
શિક્શણધિકારીને : શિક્શણ
શિક્શણધિકારીએ : શિક્શણ
શિક્શણધામથી : શિક્શણ
શિક્શણકુદરતી : શિક્શણ
શિક્શણકાજે : શિક્શણ
શિક્શણઉદ્યોગની : શિક્શણ
શિક્શણશાસ્ત્રીઓમાં : શિક્શણ
શિક્શણશાખાએ : શિક્શણ
શિક્શણવેરામાંથી : શિક્શણ
શિક્શણવર્ષે : શિક્શણ
શિક્શણબોર્ડનો : શિક્શણ
શિક્શણપ્રસારની : શિક્શણ
શિક્શણપ્રથામાં : શિક્શણ
શિક્શણપ

KeyboardInterrupt: ignored

In [ ]:
l_list = [6,7,8,9,10]
alpha_list = [4,6,8,10,12]
delta_list = [0.5,0.6,0.7,0.8,0.9,1]
unique_tokens = pickle.load(open(os.path.join(SAVE_PATH,"unique_token"),'rb'))

for l in l_list:
    for alpha in alpha_list:
        for delta in delta_list:
            st = GRAS(unique_tokens,l,alpha,delta)
            print("l =",l,"    alpha =",alpha,"    delta =",delta)
            root_word = st.get_root_dir()
            print("length =",len(root_word))
            name = "root_word_dir_"+str(l)+'_'+str(alpha)+'_'+str(delta)
            print(name)
            pickle.dump( root_word, open(os.path.join(SAVE_PATH,name), "wb" ) )


l = 6     alpha = 4     delta = 0.5
generating sets...



generating suffix_pair_freq....



generating classes....



root dir ready...
length = 968373
root_word_dir_6_4_0.5
l = 6     alpha = 4     delta = 0.6
generating sets...



generating suffix_pair_freq....


...


generating classes....



root dir ready...
length = 168953
root_word_dir_10_12_0.9
l = 10     alpha = 12     delta = 1
generating sets...



generating suffix_pair_freq....



generating classes....



root dir ready...
length = 168700
root_word_dir_10_12_1
